In [1]:
import pandas as pd
import numpy as np
import requests
from time import sleep
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

In [99]:
def get_clinic_data(page_number):
    url = f"https://www.local.ch/en/q/Switzerland/clinique?page={page_number}&rid=j6VE&slot=tel"
    response = requests.get(url, timeout = 20)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        listings = soup.find_all("div", class_="card-info-element")
        addresses = soup.find_all('div', class_ = "card-info-address")
        cl_type = soup.find_all('div', class_ = "card-info-category")
        link = soup.find_all('div', "entry-card-info-element entry-card-info-element-first")
        rating = soup.find_all('div', class_ = "summary-ratings-count")
        review = soup.find_all('span', class_ = 'summary-ratings-average lui-font-bold')
        if rating:
            if review:
                return (listings, addresses, cl_type, link, rating, review)
            else:
                return (listings, addresses, cl_type, link, [], [])

        else:
            return None
    else:
        return None

page_number = 1
clinic_listings = []
clinic_addresses = []
clinic_type = []
clinic_reviews = []
clinic_ratings = []
links = []

with ThreadPoolExecutor() as executor:
    while True:
        future = executor.submit(get_clinic_data, page_number)
        result = future.result()
        if result:
            if result[0]:
                clinic_listings.extend(result[0])
                clinic_addresses.extend(result[1])
                clinic_type.extend(result[2])
                links.extend(result[3])
                clinic_reviews.extend(result[5])
                clinic_ratings.extend(result[4])

                print(f'Page {page_number} completed')

                page_number += 1
        else:
            break

print(f"Number of clinic listings: {len(clinic_listings)}")


Page 1 completed
Page 2 completed
Page 3 completed
Page 4 completed


KeyboardInterrupt: 

In [22]:
len(get_clinic_data(1))

4

In [100]:
# Initialize an empty list to store the clinic listings
print(len(clinic_listings))
print(len(clinic_addresses))
print(len(clinic_type))
print(len(clinic_reviews))
print(len(clinic_ratings))
print(len(links))

80
40
40
30
38
38


In [101]:
clinic_names = []
for names in clinic_listings:
    try:
        name_element = names.find("h2", class_="lui-margin-vertical-zero card-info-title")
        name = name_element.text.strip()
        clinic_names.append(name)
    except AttributeError:
        continue
len(clinic_names)

40

In [102]:
clinic_address = []
for address in clinic_addresses:

    address = address.text.strip()
    clinic_address.append(address)
len(clinic_address)

40

In [103]:
type_of_clinic = []
for type_ in clinic_type:
    type_of_clinic.append(type_.text.split(' • ')[0])
len(type_of_clinic) 

40

In [104]:
clinic_review = []
for rev in clinic_reviews:
    clinic_review.append(rev.text)
len(clinic_review)
# clinic_reviews.find('div', "feedback-summary-ratings-value lui-margin-bottom-xxs").find('span', class_ = "feedback-summary-ratings-average lui-font-bold").text

30

In [105]:
ratings = []
for num in clinic_ratings:
    if num.text == 'Write the first review':
        ratings.append(None)
    else:
        rating = num.text.strip().split('\n| ')
        ratings.append(rating)
len(ratings)

38

In [93]:
clinic_ratings[5].text

'Write the first review'

In [82]:
for i in clinic_ratings:
    print(i)

<div class="summary-ratings-count">
(3)
</div>
<div class="summary-ratings-count">
(2)
</div>
<div class="summary-ratings-count">
(8)
</div>
<div class="summary-ratings-count">
(6)
</div>
<div class="summary-ratings-count">Write the first review</div>
<div class="summary-ratings-count">Write the first review</div>
<div class="summary-ratings-count">
(7)
</div>
<div class="summary-ratings-count">
(4)
</div>
<div class="summary-ratings-count">
(3)
</div>
<div class="summary-ratings-count">
(2)
</div>
<div class="summary-ratings-count">Write the first review</div>
<div class="summary-ratings-count">
(5)
</div>
<div class="summary-ratings-count">
(8)
</div>
<div class="summary-ratings-count">
(1)
</div>
<div class="summary-ratings-count">
(9)
</div>
<div class="summary-ratings-count">Write the first review</div>
<div class="summary-ratings-count">
(9)
</div>
<div class="summary-ratings-count">
(4)
</div>
<div class="summary-ratings-count">
(2)
</div>
<div class="summary-ratings-count">
(9)

In [ ]:
len(clinic_names), len(clinic_address)

In [ ]:
links[248].find('a')['href']

In [ ]:
link = []
for i in range(len(links)):
    link.append(links[i].find('a')['href'])

In [ ]:
link[]

In [ ]:
df = pd.DataFrame({'Clinic_Name' : clinic_names[:250],
                 'Address':clinic_address[:250],
                 'Clinic_Type': type_of_clinic[:250],
                 'Clinic_reviews': clinic_review[:250],
                 'Links':link[:250]})

In [ ]:
df.Links[12]

In [ ]:
df[df.Clinic_reviews == '5.0'].head(20)

In [ ]:

df[ 'Links'][60]

In [44]:
def get_clinic_data(page_number):
    global all_listings, listings
    all_listings = []
    listings = []
    url = f"https://www.local.ch/en/q/Switzerland/Clinique?page={page_number}&rid=3VCM&slot=tel"
    response = requests.get(url, timeout = 20)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        listing = soup.find_all("div", class_ ="entry-card lui-padding-horizontal-m lui-padding-vertical-m entry-card-highlight entry-card-tel")
        listings.append(listing)
        for list_ in listings:
            c = list_.find_all('div', class_ = "summary-ratings-count" )
            if c:
                all_listings.append(list_)
            else:
                return None
    else:
        return None

In [39]:
for i in range(1, 9):
    get_clinic_data(i)

In [40]:
len(all_listings)

5

In [41]:
rating = []
for l in all_listings:
    aa = l.find('div', class_ = "summary-ratings-count" )
    rating.append(aa)

In [42]:
rating

[<div class="summary-ratings-count">
 (7)
 </div>,
 <div class="summary-ratings-count">
 (7)
 </div>,
 <div class="summary-ratings-count">Write the first review</div>,
 <div class="summary-ratings-count">
 (4)
 </div>,
 <div class="summary-ratings-count">
 (7)
 </div>]

In [43]:

for i in all_listings:
    print(i.find('div', class_ = "summary-ratings-count" ))

<div class="summary-ratings-count">
(7)
</div>
<div class="summary-ratings-count">
(7)
</div>
<div class="summary-ratings-count">Write the first review</div>
<div class="summary-ratings-count">
(4)
</div>
<div class="summary-ratings-count">
(7)
</div>


In [6]:
len(all_listings)

5

In [47]:
rating

[<div class="summary-ratings-count">
 (6)
 </div>,
 <div class="summary-ratings-count">
 (30)
 </div>,
 <div class="summary-ratings-count">
 (5)
 </div>,
 <div class="summary-ratings-count">Write the first review</div>]

In [16]:
clinic_names = []
for names in all_listings:
    try:
        name_element = names.find("h2", class_="lui-margin-vertical-zero card-info-title")
        name = name_element.text.strip()
        clinic_names.append(name)
    except AttributeError:
        continue
len(clinic_names)

TypeError: find() takes no keyword arguments

In [42]:
pd.DataFrame({'name':clinic_names,
              'rating':rating})

,name,rating
0,Tierklinik Rhenus AG,(6)
1,Vitadent,(30)
2,Zahnarztpraxis Embrach-Rorbas AG,(5)
3,Zahnarztpraxis Embrach-Rorbas AG,Write the first review


In [43]:
review

NameError: name 'review' is not defined

In [50]:
len(rating)

10

In [13]:
len(listings)

20

In [ ]:
soup.